# 7. Wie gut lässt sich das Ergebnis eines Matches vorhersagen? Welches sind die wichtigsten Features für eine solche Prognose?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from tabulate import tabulate

In [2]:
matches = pd.read_csv('../data/raw/atp_matches_till_2022.csv')
# players = pd.read_csv('../data/raw/atp_players_till_2022.csv')
# rankings = pd.read_csv('../data/raw/atp_rankings_till_2022.csv')

In [3]:
matches.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')

## Unnötige Features

- `Tunier Infos:` 'tourney_name' (redundant, tourney_id)
- `Gewinner Infos:` 'winner_name' (redundant, winner_id)
- `Verlierer Infos:` 'loser_name' (redundant, looser_id)
- `Spiel Infos:`
- `Spielstatistiken:` 'score', 'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced' (Spielstatistiken kann man vorher nicht wissen)
- `Rangpunkte:` 'winner_rank_points', 'loser_rank_points'

In [4]:
df = matches[['tourney_id', 'surface', 'draw_size', 'tourney_level', 'tourney_date', 'match_num', 
        'winner_id', 'winner_seed', 'winner_entry', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
        'loser_id', 'loser_seed', 'loser_entry', 'loser_hand', 'loser_ht', 'loser_ioc', 'loser_age', 
        'best_of', 'round',
        'winner_rank', 'loser_rank']]

In [5]:
df.columns

Index(['tourney_id', 'surface', 'draw_size', 'tourney_level', 'tourney_date',
       'match_num', 'winner_id', 'winner_seed', 'winner_entry', 'winner_hand',
       'winner_ht', 'winner_ioc', 'winner_age', 'loser_id', 'loser_seed',
       'loser_entry', 'loser_hand', 'loser_ht', 'loser_ioc', 'loser_age',
       'best_of', 'round', 'winner_rank', 'loser_rank'],
      dtype='object')

In [6]:
none_percent = df.isnull().sum() * 100 / len(df)
none_values_df = pd.DataFrame({'Feature name': df.columns, 'None values count': df.isnull().sum(),
                                 'None values percent': none_percent})
none_values_df.reset_index().drop(columns=['index']).sort_values(by='None values percent', ascending=False).head(10)

,Feature name,None values count,None values percent
8,winner_entry,171891,91.353150
15,loser_entry,160432,85.263152
14,loser_seed,152824,81.219806
7,winner_seed,118467,62.960443
23,loser_rank,43327,23.026557
22,winner_rank,34964,18.581959
17,loser_ht,28698,15.251832
10,winner_ht,16237,8.629312
19,loser_age,4825,2.564293
1,surface,2317,1.231392


In [7]:
df = df.drop(columns=['winner_seed', 'winner_entry', 'loser_seed', 'loser_entry'])

In [8]:
# Convert the tourney_date column to a string
df['tourney_date'] = df['tourney_date'].astype(str)

# Create new columns for year, month, and day
df['year'] = df['tourney_date'].str[:4].astype(int)
df['month'] = df['tourney_date'].str[4:6].astype(int)
df['day'] = df['tourney_date'].str[6:].astype(int)
df = df.drop(columns=['tourney_date'])

In [9]:
from sklearn.calibration import LabelEncoder
from sklearn.preprocessing import LabelBinarizer

hand_encoder = LabelEncoder()
df['winner_hand'] = hand_encoder.fit_transform(df['winner_hand'].astype(str))
df['loser_hand'] = hand_encoder.transform(df['loser_hand'].astype(str))

df['winner_ioc'] = LabelEncoder().fit_transform(df['winner_ioc'].astype(str))
df['loser_ioc'] = LabelEncoder().fit_transform(df['loser_ioc'].astype(str))

df['surface'] = LabelBinarizer().fit_transform(df['surface'].astype(str))
df['tourney_level'] = LabelEncoder().fit_transform(df['tourney_level'].astype(str))
df['tourney_id'] = LabelEncoder().fit_transform(df['tourney_id'].astype(str))
df['round'] = LabelEncoder().fit_transform(df['round'].astype(str))

In [10]:
# Erstellen Sie Kopien der ursprünglichen Spalten, falls Sie sie später benötigen
df['original_winner_id'] = df['winner_id']
df['original_loser_id'] = df['loser_id']

# Sortieren Sie die Personen in jeder Zeile nach Rang
df['person1_id'], df['person2_id'] = np.where(df['winner_rank'] > df['loser_rank'], (df['loser_id'], df['winner_id']), (df['winner_id'], df['loser_id']))

# Wiederholen Sie diesen Vorgang für alle anderen Spalten
columns_to_update = ['hand', 'ht', 'ioc', 'age', 'rank']
for column in columns_to_update:
    df[f'person1_{column}'], df[f'person2_{column}'] = np.where(df['winner_rank'] > df['loser_rank'], (df[f'loser_{column}'], df[f'winner_{column}']), (df[f'winner_{column}'], df[f'loser_{column}']))

df['person1_won'] = np.where(df['person1_id'] == df['original_winner_id'], 1, 0)

In [11]:
# df[['person1_won', 'winner_id', 'loser_id', 'winner_rank', 'loser_rank', 'person1_id', 'person2_id', 'person1_rank', 'person2_rank']].dropna()

In [12]:
columns_to_drop = ['winner_id', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age', 'winner_rank',
                   'loser_id', 'loser_hand', 'loser_ht', 'loser_ioc', 'loser_age', 'loser_rank']

df.drop(columns=columns_to_drop, inplace=True)

In [13]:
df = df.fillna(df.median())

In [14]:
df

,tourney_id,surface,draw_size,tourney_level,match_num,best_of,round,year,month,day,...,person2_hand,person1_ht,person2_ht,person1_ioc,person2_ioc,person1_age,person2_age,person1_rank,person2_rank,person1_won
0,1,0,32,0,270,3,6,1968,7,8,...,2,185.0,185.0,5,130,25.4,24.0,33.0,86.0,1
1,1,0,32,0,271,3,6,1968,7,8,...,2,185.0,185.0,96,59,25.4,25.2,33.0,86.0,1
2,1,0,32,0,272,3,6,1968,7,8,...,2,185.0,185.0,53,59,25.4,25.2,33.0,86.0,1
3,1,0,32,0,273,3,6,1968,7,8,...,2,178.0,185.0,80,141,24.3,25.2,33.0,86.0,1
4,1,0,32,0,274,3,6,1968,7,8,...,2,185.0,185.0,121,59,21.3,25.2,33.0,86.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188156,8113,0,4,1,4,3,8,2022,3,4,...,1,185.0,188.0,44,133,20.8,20.2,1103.0,1130.0,1
188157,8113,0,4,1,5,3,8,2022,3,4,...,1,185.0,185.0,44,63,21.5,28.0,808.0,1390.0,1
188158,8114,0,4,1,1,3,8,2022,3,4,...,2,185.0,185.0,48,12,23.4,33.0,1059.0,1881.0,1
188159,8114,0,4,1,2,3,8,2022,3,4,...,1,185.0,185.0,48,12,17.7,21.7,33.0,86.0,1


In [15]:
df.columns

Index(['tourney_id', 'surface', 'draw_size', 'tourney_level', 'match_num',
       'best_of', 'round', 'year', 'month', 'day', 'original_winner_id',
       'original_loser_id', 'person1_id', 'person2_id', 'person1_hand',
       'person2_hand', 'person1_ht', 'person2_ht', 'person1_ioc',
       'person2_ioc', 'person1_age', 'person2_age', 'person1_rank',
       'person2_rank', 'person1_won'],
      dtype='object')

In [16]:
y = df['person1_won']
X = df[['tourney_id', 'surface', 'draw_size', 'tourney_level', 'match_num',
       'best_of', 'round', 'year', 'month', 'day', 'person1_id', 'person2_id', 'person1_hand',
       'person2_hand', 'person1_ht', 'person2_ht', 'person1_ioc',
       'person2_ioc', 'person1_age', 'person2_age', 'person1_rank', 'person2_rank']]

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [18]:
from sklearn.ensemble import RandomForestClassifier

#Call the classifier
RF_classifier = RandomForestClassifier(random_state=42)
#fit the data
RF_classifier.fit(X_train, y_train)
#predict 
RF_predictions = RF_classifier.predict(X_test)

In [19]:
print(f"Accuracy Trainingsdaten: {RF_classifier.score(X_train, y_train)}")
print(f"Accuracy Testdaten: {RF_classifier.score(X_test, y_test)}")

Accuracy Trainingsdaten: 1.0
Accuracy Testdaten: 0.9368107777748252


Vorhandene Informationen:

- `Turnierinformationen`: Name, Oberfläche (Gras, Hartplatz, Sand, Teppich), Größe des Teilnehmerfeldes, Turnierlevel und -datum.
- `Matchdetails`: Spielnummer, IDs und Set-Ergebnisse der Spieler.
- `Spielerinformationen`: Setzplatz, Eintrittsmethode, Herkunftsland, Hand (Rechts-/Linkshänder), Größe, Geburtsdatum.
- `Spielstatistiken`: Erste und zweite Aufschlagprozente, Breakpoints, Asse, Doppelfehler usw.
- `Rankinginformationen`: ATP-Rang und Ranking-Punkte der Spieler.

Zusätzliche Informationen könnten hinzugezogen werden:

- `Spieler-Statistiken:` Dies beinhaltet aktuelle Weltranglistenpositionen, frühere Leistungen, Sieg-Niederlage-Statistiken, Performanz auf verschiedenen Belägen (Hartplatz, Rasen, Sand), etc.
- `Aktuelle Form`: Leistungen in den letzten Spielen oder Turnieren können ein Indikator für die aktuelle Form eines Spielers sein.
- `Head-to-Head-Bilanz`: Die vergangenen Aufeinandertreffen zwischen den Spielern können Aufschluss über psychologische Vorteile geben.
- `Physiologische und psychologische Faktoren`: Alter, Fitness, Verletzungen, mentale Stärke unter Druck, etc.
- `Umgebungsbedingungen`: Wetter, Belag, Höhe des Spielortes und Heimvorteil können ebenfalls eine Rolle spielen.

Zusammenfassung:

(mit großen Abstand) Wichtigstes Feature: Spieler-Ranking und -Punkte: Höher gerankte Spieler gewinnen tendenziell häufiger.

- Head-to-Head-Statistiken: Wenn zwei Spieler zuvor gegeneinander gespielt haben, können diese Ergebnisse aufschlussreich sein.
- Spielstatistiken: Asse, Doppelfehler, Gewinnquote bei erstem Aufschlag usw.
- Oberflächenpräferenz: Manche Spieler sind auf bestimmten Oberflächen stärker.
- Aktuelle Form: Ergebnisse der letzten Matches können die aktuelle Spielstärke widerspiegeln.
- Turnierlevel und -geschichte: Leistung in bestimmten Turnieren oder Turnierstufen (z.B. Grand Slam vs. ATP 250).